# Part 1: Tensorflow Mechanics 

Tensorflow has various high level APIs that allow you to build simple machine learning models and algorithms with just very few lines of code, by hiding from the user what's happening in the backround.
This can save a lot of time if you are already familiar with tensorflow. 
<br>
However, if you want to develop new models and algorithms, it is important to understand the underlying mechanics.
Let's start from the bottom up!

You are probably familiar with numpy and know that computations are executed as soon as the python interpreter executes that line of code. This is not the case in tensorflow! (Note: Eager execution will be covered later)
<bl>
The 2 core concepts of Tensorflow are symbolic dataflow **Graphs** and **Sessions**. 
 - The **graph** represents the dataflow in your model in terms of mathematical **operations** between **tensors**. 
 When *defining* the graph, the actual *computation* is not yet executed. 
 This is in contrast to imperative programming (such as python itself, e.g. with numpy)
 - A **session** is the interface between your program in which the symbolic graph was defined, and the C++ runtime (backend). It enables running the graph.

In [1]:
import tensorflow as tf

## Operations and Tensors

The following code snippet creates 3 operations which will be automatically registered to the *default* graph.

In [2]:
a = tf.constant(0.5, dtype=tf.float32, name="a")
b = tf.constant(-4.0)  # default dtype is float32
c = tf.multiply(a, b) 

The function tf.constant builds an **operation** that creates a constant when run in a **session**. It also returns a symbolic **tensor** that can be used for further graph building.
<br>
tf.multiply takes 2 **tensors** as inputs, builds the **operation** that will multiply these tensors when run in a session, and returns a symbolic **tensor**.

In [3]:
# Let's inspect the tensors
print(a, b, c, "\n")
print(a.dtype, b.dtype, c.dtype, "\n")
print(a.shape, b.shape, c.shape, "\n")
print(a.name, b.name, c.name, "\n")

Tensor("a:0", shape=(), dtype=float32) Tensor("Const:0", shape=(), dtype=float32) Tensor("Mul:0", shape=(), dtype=float32) 

<dtype: 'float32'> <dtype: 'float32'> <dtype: 'float32'> 

() () () 

a:0 Const:0 Mul:0 



As can be seen the variable "c" was not actually computed yet. Let's contrast this with numpy

In [4]:
import numpy as np
x = np.array(0.5, dtype=np.float32)
y = np.array(-4.0, dtype=np.float32)  
z = np.multiply(x, y)
print(x, y, z)

0.5 -4.0 -2.0


## Graphs

The functions tf.constant and tf.multiply automatically register the respective **operation** to the current default graph.
<br>
**Note**: There is always a default graph. 

In [5]:
default_graph = tf.get_default_graph()

In [6]:
# Let's inspect the default graph and its registered operations
print(default_graph.get_operations(), "\n")
for op in default_graph.get_operations():
      print(op.values())

[<tf.Operation 'a' type=Const>, <tf.Operation 'Const' type=Const>, <tf.Operation 'Mul' type=Mul>] 

(<tf.Tensor 'a:0' shape=() dtype=float32>,)
(<tf.Tensor 'Const:0' shape=() dtype=float32>,)
(<tf.Tensor 'Mul:0' shape=() dtype=float32>,)


We can also create a new graph manually and set it as the default graph. This is rarely needed, but it certainly helps to understand what tensorflow is doing in the background.
<br>
The following code creates a new graph, sets it as the default graph within the context block, and registers a few operations to it. 

In [7]:
graph_1 = tf.Graph()  # Construct a new graph
with graph_1.as_default():  # default in this context block
    a_1 = tf.constant(-1.0, name="a_1")  # registers operation that constructs a (constant) tensor
    b_1 = tf.constant(1.0, name="b_1")  # also returns a symbolic tensor
    c_1 = tf.multiply(a_1,b_1, name="c_1")  # registers multiply operation and returns symbolic tensor

In [8]:
graph_2 = tf.Graph()
with graph_2.as_default():
    print("graph_2 is default graph within context: {}".format(graph_2 is tf.get_default_graph())) 
    a_2 = tf.constant(-2.0, )
    b_2 = tf.constant(2.0)
    c_2 = tf.multiply(a_2, b_2)
print("graph_2 is default graph after context: {}".format(graph_2 is tf.get_default_graph())) 

graph_2 is default graph within context: True
graph_2 is default graph after context: False


Let's inspect our new graphs and verify that the variables in the "with" context are registered to the correct graph. 

In [9]:
for op in graph_1.get_operations():
      print(op.values())

(<tf.Tensor 'a_1:0' shape=() dtype=float32>,)
(<tf.Tensor 'b_1:0' shape=() dtype=float32>,)
(<tf.Tensor 'c_1:0' shape=() dtype=float32>,)


Our **tensors** will also have a handle to their respective graph (to which they were registered). 

In [10]:
print(a_1.graph is graph_1, b_1.graph is graph_1, c_1.graph is graph_1)
print(a_1.graph is graph_2)
print(a_1.graph is tf.get_default_graph())

True True True
False
False


**Note**: Tensorflow will keep adding new operations with unique names (appending "_NUM"), when you execute the functions (tf.constant, tf.multiply, etc.) multiple times.
When prototyping with jupyter notebooks, a useful function is: tf.reset_default_graph()

# Sessions

In [11]:
sess = tf.Session()  

In [12]:
print(sess.run(c))
print(sess.run([a, b, c]))
print(sess.run({"a":a, "b":b, "c":c}))

-2.0
[0.5, -4.0, -2.0]
{'a': 0.5, 'b': -4.0, 'c': -2.0}


When we create a session, it will be bound to the current default graph.

In [13]:
try:
    print(sess.run(c_1))  # c_1 was registered at graph_1
except:
    print("Cannot run c_1 from current (default) graph.")
with graph_1.as_default():
    tmp_sess = tf.Session()
    print(tmp_sess.run({"c_1": c_1}))

Cannot run c_1 from current (default) graph.
{'c_1': -1.0}


# Placeholders

Placeholders are tensors, associated with an operation that replaces the placeholder with actual data when it is **run** using **session**. 

In [14]:
a_ph = tf.placeholder(shape=[], dtype=tf.float32, name="a_ph")  # [] means scalar (tensor of rank 0)
b_ph = tf.placeholder(shape=[], dtype=tf.float32, name="b_ph")
c_from_ph = tf.multiply(a_ph, b_ph)

In [15]:
print(a_ph, b_ph)

Tensor("a_ph:0", shape=(), dtype=float32) Tensor("b_ph:0", shape=(), dtype=float32)


In [16]:
# try:
#     result = sess.run(c_from_ph)  # This will not work, we must provide values for a_ph and b_ph
# except TypeError:
#     print(e)

In [17]:
feed_dict = {a_ph:2.0, b_ph:np.pi}  # {placeholder: data}, where data can be e.g. native python or numpy arrays.
result = sess.run(c_from_ph, feed_dict=feed_dict)
print(result)

6.2831855


# Variables

In tensorflow, the tf.Variable class represents shared, persistent state that exists between individual sess.run() calls. 
<br>
For example, the weights and biases in neural networks or linear regression are stored as variables.
<br>
As variables live outside sess.run(), they must be initialized (once), before any operation that requires them can be evaluated.

In [18]:
batch_size = 10  # tf operations are broadcasted along the first dimension (batch).
dim_in, dim_out = 5, 1

In [19]:
inpt = tf.placeholder(shape=[batch_size, dim_in], dtype=tf.float32, name="input")
weights = tf.get_variable(shape=[dim_in, dim_out], name="weight")
bias = tf.get_variable(shape=[dim_out], name="bias")
outpt = tf.add(bias, tf.matmul(inpt, weights))

In [20]:
init_all_vars_op = tf.global_variables_initializer()  # == tf.variables_initializer(tf.global_variables())
sess.run(init_all_vars_op)

In [21]:
batch = np.random.rand(batch_size, dim_in)
feed_dict = {inpt: batch}
result = sess.run(outpt, feed_dict=feed_dict)

In [23]:
print(result)

[[ 0.73169905]
 [ 0.00094247]
 [ 0.36830303]
 [ 0.7220247 ]
 [ 0.22914588]
 [ 0.05983466]
 [ 0.29493755]
 [-0.15804785]
 [ 0.286295  ]
 [-0.30874217]]


# Short Recap 

The **Graph** comprises Operations, Tensors, Placeholders, Variables, as well as metadata.
<br>
Functions such as tf.constant, tf.placeholder, tf.matmul, tf.add, etc. register **operations** to the default graph and return a **tensor**. 
Tensors are symbols, not actual data. The actual data lives only within the run() call of a **session**.
<br>
**Variables**, obtained by tf.get_variable, are persistent and must be initialized.
Use variables for parameters of parametric machine learning models. 